# Setup

In [7]:
import Pkg

In [8]:
Pkg.activate(".")

  Activating environment at `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Project.toml`


In [9]:
# Pkg.add("BenchmarkTools")
# Pkg.add("CUDA")
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Project.toml`
  No Changes to `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Manifest.toml`


In [10]:
using CUDA
using Distributions
using BenchmarkTools

# Create graph

In [11]:
N = 16

# graph = zeros(2^N)
graph = rand(Uniform(-5,5),N, N)
graph = graph * graph'

qubo = zeros(N,N)
constant = 0

for i in 1:N
    qubo[i,i] = 2 * graph[i,i]
    constant += graph[i,i]
    
    for j in 1:N
        if i!=j
            low, high = sort!([i, j])
            constant -= graph[low,high]*0.5
            qubo[i,i] -= 2*graph[low,high]
            qubo[low,high] += graph[low,high]*2
        end
    end
end


cu_graph = graph |> cu 
cu_qubo = qubo |> cu 

16×16 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 254.719  -111.542  -113.888   …   237.684     148.818     -94.5745
   0.0     403.513   147.061      -226.27      185.143     103.62
   0.0       0.0     -47.3911       82.6755    155.325      29.4052
   0.0       0.0       0.0          79.3011     72.3483     35.4507
   0.0       0.0       0.0         -10.1123    -42.2221    112.01
   0.0       0.0       0.0     …   -45.5585   -186.122     -18.0521
   0.0       0.0       0.0          40.0203    -65.8839    265.256
   0.0       0.0       0.0          17.0092    283.356      10.1853
   0.0       0.0       0.0          35.7524    -95.7432   -100.048
   0.0       0.0       0.0         122.004     -75.2388    -27.9586
   0.0       0.0       0.0     …     4.57685     5.16146   -70.1653
   0.0       0.0       0.0           2.56107   -23.473    -102.808
   0.0       0.0       0.0         149.016     128.529     131.988
   0.0       0.0       0.0        -134.627     126.724      48.6315
   0.0

# Brute forece

In [22]:
function energy(graph, state_code)
    F = 0
    N = size(graph)[1]
    q = digits(state_code, base=2, pad=N) #|> reverse

    for i in 1:N
        F -= graph[i,i]*q[i]  
        for j in 1:N
            low, high = sort!([i, j])
            F -= graph[low,high]*q[i]*q[j]
        end
    end
    return F
end

@btime begin
    res = zeros(2^N)

    for k in 1:2^N
        F = energy(qubo, k)
        res[k] = F
    end

    sort!(res)
end

  892.530 ms (17103879 allocations: 1.52 GiB)


65536-element Vector{Float64}:
 -8916.608333408969
 -8782.341466146356
 -8689.73401701808
 -8645.567010164743
 -8621.010655115255
 -8572.312065209235
 -8535.484878155952
 -8395.557953480908
 -8387.483242725897
 -8362.831542049069
 -8324.197687490341
 -8322.52923673931
 -8322.505953996846
     ⋮
    79.9987487650198
    94.78218288741475
    95.8977913670762
   100.40478032197018
   160.3009925135584
   176.10598593444325
   198.68450109954915
   202.04369077705428
   208.6949747361241
   263.24134270843865
   269.2532174028833
   312.16512374425565

# Brute-force GPU

In [23]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel(qubo, energies)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]

    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
          
    return
end

function main()
    k = 8
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel(cu_qubo, energies)
   
    states = sortperm(energies)
    energies[states[1:10]]
    
#     sort!(energies)
    
end

@btime begin
    main()
end

# main()

  454.303 μs (2464 allocations: 611.23 KiB)


10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -8916.608
 -8782.343
 -8689.737
 -8645.568
 -8621.012
 -8572.3125
 -8535.485
 -8395.559
 -8387.483
 -8362.833

In [31]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel2(qubo, energies, part_lst, part_st)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]

    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
    
    sync_threads()
    
    if j == 1
        k = (i - 1) * blockDim().x + 1
        n = blockDim().x
        nr_of_block = gridDim().x
        
#         for ii in k:k+n-1
#             value = low_en[ii+1]
#             jj = ii
#             while jj > 0 && low_en[jj] > value
#                 low_en[jj+1] = low_en[jj]
#                 jj -= 1
#             end
#             low_en[i] = value
#         end 
        
        value=energies[k]
        st=k
        for ii in k:k+n-1
            if value > energies[ii]
                value = energies[ii]
                st=k
            end
        end 
               
        sync_threads()
        
        part_lst[i] = value # low_en[k]
        part_st[i] = st
    end
    
    return
end

function main()
    k = 4
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    part_st = CUDA.zeros(2^(N-k))
    part_lst = CUDA.zeros(2^(N-k))
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel2(cu_qubo, energies, part_lst, part_st)
   
    states = sortperm(part_lst)
    part_lst[states]
    
#     sort!(part_lst)
    
end

@btime begin
    main()
end

# main()

  384.412 μs (1096 allocations: 77.45 KiB)


4096-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -8916.608
 -8782.343
 -8689.737
 -8645.568
 -8621.012
 -8572.3125
 -8535.485
 -8395.559
 -8362.833
 -8322.505
 -8249.573
 -8243.767
 -8241.081
     ⋮
 -1992.5063
 -1985.6239
 -1975.7223
 -1921.5736
 -1896.0372
 -1875.0636
 -1812.8264
 -1791.4513
 -1678.2026
 -1610.064
 -1584.5175
 -1520.9233

In [30]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel2(qubo, energies, part_lst, part_st)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]
    K = size(part_lst)[1]
    
    block_per_pe = Int(ceil(gridDim().x/K))
    
    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
    
    sync_threads()
    
    if j == 1
        k = (i - 1) * blockDim().x + 1
        n = blockDim().x
    
        value=energies[k]
        st=k
        for ii in k:k+n-1
            if value > energies[ii]
                value = energies[ii]
                st=k
            end
        end 

        pi = Int(ceil(i/block_per_pe))
        
        threadfence()
        if value < part_lst[pi]
            @inbounds part_lst[pi] = value # low_en[k]
            @inbounds part_st[pi] = st
        end

    end 
       
    return
end

function main()
    k = 4
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    part_st = CUDA.zeros(10)
    part_lst = CUDA.zeros(10)
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel2(cu_qubo, energies, part_lst, part_st)
   
#     states = sortperm(energies)
#     energies[states]
    
#     sort!(part_lst)
 
    states = sortperm(part_lst)
    part_lst[states]

    
end

# @btime begin
#     main()
# end

main()

10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -8782.343
 -8322.505
 -7632.4463
 -7560.2656
 -7420.807
 -7357.5923
 -7355.502
 -7033.1484
 -6190.993
 -5633.304

In [28]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel2(qubo, energies, part_lst, part_st)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]
    K = size(part_lst)[1]
    
    block_per_pe = Int(ceil(gridDim().x/K))
    
    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
    
    sync_threads() 
    threadfence_system()
    
    for ii in 1:K
        if part_lst[ii] > F
            for jj in ii+1:K
                @inbounds part_lst[K+ii+1-jj]=part_lst[K+ii+1-jj-1]
            end
            @inbounds part_lst[ii] = F
            @inbounds part_st[ii] = state_code
        end
    end 
    
#     if j == 1
#         k = (i - 1) * blockDim().x + 1
#         n = blockDim().x

#         value=energies[k]
#         st=k
#         for ii in k:k+n-1
#             if value > energies[ii]
#                 value = energies[ii]
#                 st=k
#             end
#         end 

# #         for ii in 1:K
# #             if part_lst[ii] > value
# #                 for jj in ii+1:K
# #                     @inbounds part_lst[K+ii+1-jj]=part_lst[K+ii+1-jj-1]
# #                 end
# #                 @inbounds part_lst[ii] = value
# #                 @inbounds part_st[ii] = st
# #             end
# #         end 
        
#         pi = Int(ceil(i/block_per_pe))

#         threadfence()
#         if value < part_lst[pi]
#             @inbounds part_lst[pi] = value # low_en[k]
#             @inbounds part_st[pi] = st
#         end
        
#     end  
       
    return
end

# function arginsertsort!(A::Array{T}) where T <: Number
#     ids = collect(1:length(A))
#     B = copy(A)
    
#     for i in 1:length(B)-1
#         value = B[i+1]
#         ival = ids[i+1]
#         j = i
#         while j > 0 && B[j] > value
#             B[j+1] = B[j]
#             ids[j+1] = ids[j]
#             j -= 1
#         end
#         B[j+1] = value
#         ids[j+1] = ival
#     end 
#     return ids
# end

function main()
    k = 4
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    part_st = CUDA.zeros(10)
    part_lst = CUDA.zeros(10)
    
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel2(cu_qubo, energies, part_lst, part_st)
   
#     states = sortperm(energies)
#     energies[states]
    
#     sort!(part_lst)
 
    states = sortperm(part_lst)
    
    part_lst[states]

    
end

# @btime begin

#     main()
# end

main()

10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737
 -8689.737

In [17]:
n = 10

k = 4
block_per_pe = Int(ceil(2^(N-k)/n))
2^(N-k)

4096

In [18]:
2^(N-k)

4096

In [19]:
n * block_per_pe

4100

In [20]:
538/block_per_pe

1.3121951219512196

In [21]:
Int(ceil(538/block_per_pe))

2

In [599]:
partialsort!([2,-5,3,1,5,21,-53], 1:4)

4-element view(::Vector{Int64}, 1:4) with eltype Int64:
 -53
  -5
   1
   2

In [591]:
sortperm!([2,-5,3,1,5,21,-53])

LoadError: MethodError: no method matching sortperm!(::Vector{Int64})
[0mClosest candidates are:
[0m  sortperm!(::AbstractVector{var"#s829"} where var"#s829"<:Integer, [91m::AbstractVector{T} where T[39m; alg, lt, by, rev, order, initialized) at sort.jl:949
[0m  sortperm!([91m::AnyCuArray{T, N} where N[39m, [91m::AnyCuArray{T, N} where {T, N}[39m; initialized, kwargs...) where T at /home/dk/.julia/packages/CUDA/bki2w/src/sorting.jl:965

In [ ]:
gra